In [114]:
import pandas as pd
import untangle # xml
import requests # json
import re # regular expressions

In [115]:
# Configuration
our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]

In [116]:
xml = untangle.parse('https://publicdashacc.blob.core.windows.net/publicdata?restype=container&comp=list')

In [117]:
xml

Element(name = None, attributes = None, cdata = )

In [118]:
blobs = xml.EnumerationResults.Blobs.Blob

In [119]:
for blob in blobs:
    print(blob.Name.cdata)

countries.geojson
data_20200409.json
data_202004091537.json
data_202004101527.json
data_202004111452.json
data_202004121411.json
data_202004131413.json
data_202004141435.json
data_202004141544.json
data_202004151454.json
data_202004161444.json
regions.geojson
utlas.geojson


In [120]:
jsons = sorted(
    list(
        filter(
            lambda b: re.search('^data_[0-9]+.json$',b),
            map(
                lambda b: b.Name.cdata,
                blobs
            )
        )
    )
)

latest_json = jsons[-1]
print(latest_json)

data_202004161444.json


In [121]:
host = 'https://c19pub.azureedge.net/'
latest_json_url = host + latest_json
print(latest_json_url)

https://c19pub.azureedge.net/data_202004161444.json


In [122]:
json = requests.get(latest_json_url).json()

In [169]:
new_data = []
for (code, data) in json['utlas'].items():
    if code in our_utla_codes:
        for day in data['dailyConfirmedCases']:
            new_data.append([
                code,
                data['name']['value'],
                day['date'],
                day['value']
            ])
df = pd.DataFrame(data=new_data, columns=['code','name','date','value'])

In [170]:
df

,code,name,date,value
0,E08000025,Birmingham,2020-03-01,1
1,E08000025,Birmingham,2020-03-02,1
2,E08000025,Birmingham,2020-03-05,1
3,E08000025,Birmingham,2020-03-08,1
4,E08000025,Birmingham,2020-03-09,1
...,...,...,...,...
368,E10000034,Worcestershire,2020-04-11,37
369,E10000034,Worcestershire,2020-04-12,22
370,E10000034,Worcestershire,2020-04-13,7
371,E10000034,Worcestershire,2020-04-14,1
